***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [3. Positional Astronomy](../3_Positional_Astronomy/3_0_Introduction.ipynb)  
    * Previous: [3. Positional Astronomy](../3_Positional_Astronomy/3_0_Introduction.ipynb) 
    * Next: [3.2 Hour Angle (HA) and Local Sidereal Time (LST)](../3_Positional_Astronomy/3_2_Hour_Angle.ipynb)

***

Import standard modules:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Import section specific modules:

In [1]:
from IPython.core.display import HTML
styles = open("../style/course.css", "r").read()
HTML(styles)

## 3.2 Equatorial Coordinates (RA,DEC)

We can use a geographical coordinate system to uniquely identify a position on earth. We normally use the coordinates *latitude* $L_a$ (to measure north and south) and *longitude* $L_o$ (to measure east and west) to accomplish this. The equatorial coordinate system is  depicted in [Fig. 3.1 &#10549;](#pos:fig:geo).

<img src='figures/geo.png' width=50%>

<div align="center">**Figure 3.1**: The geographical coordinates latitude $L_a$ and longitude $L_o$. </div> <a id='pos:fig:geo'></a> <!--\label{pos:fig:geo}-->

We also require a coordinate system to map the stars. For all intents and purposes we may think of our universe as being projected onto a sphere of arbitrary radius. This sphere surrounds the Earth and is known as the *celestial sphere*. This is not a true representation of our universe, but it is a very useful astronomical construct. The *celestial equator* is obtained by projecting the equator of the earth onto the celestial sphere. The stars themselves do not move on the celestial sphere and therefore have a unique location on it (at least not much in one generation). The sun is an exception, it changes position in a periodic fashion during the year. The path it traverses on the celestial sphere is known as the *ecliptic*.

<p class=conclusion>
  <font size=4> **The NCP and SCP**</font>
  <br>
  <br>
&bull; The *north celestial pole* (NCP) is an important location on the celestial sphere and is obtained by projecting the north pole of the earth onto the celestial sphere. The star *Polaris* is very close to the NCP and serves as a reference when positioning a telescope. <br><br>
&bull; The *south celestial pole* (SCP) is obtained in a similar way. The imaginary circle known as the *celestial equator* is in the same plane as the equator of the earth and is obtained by projecting the equator of the earth onto the celestial sphere. The southern hemisphere
counterpart of Polaris is *Sigma Octanis*. <br><br> 
</p>

We use a specific point on the celestial equator from which we measure the location of all other celestial objects. This point is known as the *first point of Aries* ($\gamma$) <!--\vernal--> or the *vernal equinox*. The *vernal equinox* is the point where 
the ecliptic intersects the celestial equator (south to north). We discuss the vernal equinox in more detail in [$\S$ 3.2.2 &#10142;](../3_Positional_Astronomy/3_2_Hour_Angle.ipynb#pos:sec:lst) <!--\ref{pos:sec:lst}-->.  

We use the *equatorial coordinates* to uniquely identify the location of celestial objects rotating with the celestial sphere around the SCP/NCP axis.

<p class=conclusion>
  <font size=4> **Equatorial Coordinates**</font>
  <br>
  <br>
&bull; the *Right Ascension* $\alpha$ - We define the *hour circle* of an object as the circle on the celestial sphere that crosses the NCP and the object itself, while also perpendicularly intersecting with the celestial equator. The right ascension of an object is the angular distance between the vernal equinox and the hour circle of a celestial object measured along the celestial equator and is measured eastward. It is measured in **Hours Minutes Seconds** (e.g. $\alpha = 03^\text{h}13^\text{m}32.5^\text{s}$) and spans 360$\circ$ on the celestial sphere from $\alpha = 00^\text{h}00^\text{m}00^\text{s}$ (the coordinates of $\gamma$) to $\alpha = 23^\text{h}59^\text{m}59^\text{s}$. <br><br>
&bull; the *Declination* $\delta$ - the declination of an object is the angular distance from the celestial equator measured along its hour circle (it is positive in the northern celestial hemisphere and negative in the southern celestial hemisphere). It is measured in **Degrees Arcmin Arcsec** (e.g. $\delta = -15^\circ23'44''$) which spans from $\delta = -90^\circ00'00''$ (SCP) to $+\delta = 90^\circ00'00''$ (NCP). <br><br> 
</p>

The equatorial coordinates are presented graphically in [Fig. 3.2 &#10549;](#pos:fig:equatorial_coordinates) <!--\ref{pos:fig:equatorial_coordinates}-->. 

<div class=warn>
**Warning:** As for any spherical system, the Right Ascension of the NCP ($\delta=+90^ \circ$) and the SCP ($\delta=-90^ \circ$) are ill-defined. And a source close to the any CP can have an unintuitive Right Ascension.
</div>

<img src='figures/equatorial.png' width=40%>

<div align="center">**Figure 3.2**: The equatorial coordinates $\alpha$ and $\delta$. The vernal equinox $\gamma$, the equatorial reference point is also depicted. The vernal 
equinox is the point where the ecliptic (the path the sun traverses over one year) intersects the celestial equator.</div> <a id='pos:fig:equatorial_coordinates'></a> <!--\label{pos:fig:equatorial_coordinates}-->

<div class=warn>
**Warning:** One arcminute of the declination axis (e.g. $00^\circ01'00''$) is not equal to one *minute* in right ascension axis (e.g. $00^\text{h}01^\text{m}00^\text{s}$). <br>
Indeed, in RA, the 24$^\text{h}$ circle is mapped to a 360$^\circ$ circle meaning that 1 hour spans over a section of 15$^\circ$. And as 1$^\text{h}$ is 60$^\text{m}$, therefore 1$^\text{m}$ in RA correspond to $1^\text{m} = \frac{1^\text{h}}{60}=\frac{15^\circ}{60}=0.25'$. <br>
You should be careful about this **factor of 4 difference between RA min and DEC arcmin** (i.e. $\text{RA} \; 00^\text{h}01^\text{m}00^\text{s}\neq \text{DEC} \; 00^\circ01'00''$)
</div>

***

Next: [3.2 Hour Angle (HA) and Local Sidereal Time (LST)](../3_Positional_Astronomy/3_2_Hour_Angle.ipynb)